In [1]:
import findspark
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import Row
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.streaming import StreamingContext

In [2]:
#Initialisation de findspark
findspark.init()

In [3]:
spark = (SparkSession.builder.appName("Kafka Pyspark Streaming Learning")
        .master("local[*]")
        .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0")
        .getOrCreate())
spark.sparkContext.setLogLevel("ERROR")

In [11]:
KAFKA_TOPIC_NAME = "vlib_status"
KAFKA_BOOTSTRAP_SERVER = "51.38.185.58:9092"
    
df = (
    spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVER)
    .option("subscribe", KAFKA_TOPIC_NAME)
    .option("startingOffsets", "latest")
    .load()
    .selectExpr("CAST(value as STRING)")
)


jschema = schema_of_json('{a: 1}', {'allowUnquotedFieldNames':'true'})

df2 = df.select((from_json(col("value")), explode(df.value)).cast("string"), schema= jschema).(alias("parsed_value"))

query = df2 \
.writeStream \
.format("json") \
.option("path", "velib_data.json") \
.option("checkpointLocation", "checkpoint")\
.start()

query.awaitTermination()

StreamingQueryException: org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Ljava/lang/String;I)Z
=== Streaming Query ===
Identifier: [id = 53d53209-4704-4eb1-ba15-8a0f08856374, runId = 95b780ba-4938-444a-af34-77fd1d2c167f]
Current Committed Offsets: {}
Current Available Offsets: {}

Current State: ACTIVE
Thread State: RUNNABLE

Logical Plan:
Project [cast(value#80 as string) AS value#93]
+- StreamingDataSourceV2Relation [key#79, value#80, topic#81, partition#82, offset#83L, timestamp#84, timestampType#85], org.apache.spark.sql.kafka010.KafkaSourceProvider$KafkaScan@77a37cea, KafkaV2[Subscribe[vlib_status]]


In [4]:
spark = SparkSession.builder.appName('mini_projet').master("local").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [5]:
topic = "vlib_status"
bootstrap_server = "51.38.185.58:9092"
#bootstrap_server = "127.0.0.1:9092"

kafka_df = spark.readStream \
.format("kafka") \
.option("kafka.bootstrap.servers", bootstrap_server) \
.option("subscribe", topic) \
.option("includeHeaders", "true") \
.option("startingOffsets", "latest") \
.load()

#option("checkpointLocation", "/tmp/checkpoint")

#kafka_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
query = kafka_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)").writeStream.outputMode("complete").format("console").start()

query.awaitTermination()
#kafka_df.show(5)

Py4JJavaError: An error occurred while calling o38.load.
: java.lang.NoClassDefFoundError: scala/$less$colon$less
	at org.apache.spark.sql.kafka010.KafkaSourceProvider.org$apache$spark$sql$kafka010$KafkaSourceProvider$$validateStreamOptions(KafkaSourceProvider.scala:338)
	at org.apache.spark.sql.kafka010.KafkaSourceProvider.sourceSchema(KafkaSourceProvider.scala:71)
	at org.apache.spark.sql.execution.datasources.DataSource.sourceSchema(DataSource.scala:236)
	at org.apache.spark.sql.execution.datasources.DataSource.sourceInfo$lzycompute(DataSource.scala:118)
	at org.apache.spark.sql.execution.datasources.DataSource.sourceInfo(DataSource.scala:118)
	at org.apache.spark.sql.execution.streaming.StreamingRelation$.apply(StreamingRelation.scala:34)
	at org.apache.spark.sql.streaming.DataStreamReader.loadInternal(DataStreamReader.scala:168)
	at org.apache.spark.sql.streaming.DataStreamReader.load(DataStreamReader.scala:144)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Meth